# <center> Devoir Maison \#2 - MAP 433 </center>

## Trinôme:
#### BARRÉ Théo
#### CHEN Longteng
#### COSTA ALVES FREIRE Bruno

## Application Numérique

In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import scipy.stats as sps
import pandas as pd
import statsmodels as sm
import statsmodels.formula.api as smf
from IPython.display import display, Latex
import warnings
warnings.filterwarnings('ignore')

file = "Titanic.csv"
données = pd.read_csv(file, delimiter=',')

données.drop(columns=["PassengerId", "Ticket", "Cabin", "Embarked"], inplace=True)

données["Age"].fillna(données["Age"].mean(),inplace=True)

données.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500


### 1. Déterminer le nombre et la proportion de passagers décédés.

In [2]:
series = données["Survived"]
n_décédés = series.count() - series.sum()
p_décédés = 1 - series.mean()

display(Latex(f"Nombre de décédés: ${n_décédés}$"))
display(Latex(f"Proportion de décédés: ${p_décédés:.4f} = {100*p_décédés:.1f}~\%$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### 2. Déterminer le pourcentage d'hommes et de femmes parmi les personnes décédées et les survivants. Qu'observe-t-on ?

In [3]:
df = données.groupby(["Survived"])["Sex"].apply(lambda s: pd.Series({
    "Femmes (%)": f"{s[s == 'female'].count()/s.count():.2f} %", 
    "Hommes (%)": f"{s[s == 'male'].count()/s.count():.2f} %"
}))

pd.DataFrame(df).unstack()

Sex           
         Femmes (%) Hommes (%)
Survived                      
0            0.15 %     0.85 %
1            0.68 %     0.32 %

On observe que les hommes sont la majorité parmi les victimes fatales, lorsque les femmes sont la majorité parmi les survivants. 

### 3. Reprendre cette analyse pour les différentes classes. Qu'observe-t-on ?

In [4]:
df = données[["Survived", "Sex", "Pclass"]]
df = pd.pivot_table(df, values='Sex', index=['Pclass', 'Survived'],
                    columns=['Sex'], aggfunc=len)

df = df.apply(axis=1, func=lambda s: s/s.sum())
df.apply(lambda s: s.apply(lambda s: f"{s:.2f} %"))

Sex              female    male
Pclass Survived                
1      0         0.04 %  0.96 %
       1         0.67 %  0.33 %
2      0         0.06 %  0.94 %
       1         0.80 %  0.20 %
3      0         0.19 %  0.81 %
       1         0.61 %  0.39 %

Encore une fois, on observe dans chacune des classes, la plupart des décédés sont des hommes, et la plupart des survivant sont des femmes. 

### 4. Calculer la matrice de corrélation des covariables. Qu'observe-t-on ?

In [5]:
df = données

# On conventionne "female" = 1 et "male" = 1

df = pd.get_dummies(df, columns = ["Sex", "Pclass"])

print("La matrice de corrélation des covariables est : ")
df.corr()

La matrice de corrélation des covariables est : 


,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
Survived,1.000000,-0.069809,-0.035322,0.081629,0.257307,0.543351,-0.543351,0.285904,0.093349,-0.322308
Age,-0.069809,1.000000,-0.232625,-0.179191,0.091566,-0.084153,0.084153,0.319916,0.006589,-0.281004
SibSp,-0.035322,-0.232625,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.054582,-0.055932,0.092548
Parch,0.081629,-0.179191,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.017633,-0.000734,0.015790
Fare,0.257307,0.091566,0.159651,0.216225,1.000000,0.182333,-0.182333,0.591711,-0.118557,-0.413333
Sex_female,0.543351,-0.084153,0.114631,0.245489,0.182333,1.000000,-1.000000,0.098013,0.064746,-0.137143
Sex_male,-0.543351,0.084153,-0.114631,-0.245489,-0.182333,-1.000000,1.000000,-0.098013,-0.064746,0.137143
Pclass_1,0.285904,0.319916,-0.054582,-0.017633,0.591711,0.098013,-0.098013,1.000000,-0.288585,-0.626738
Pclass_2,0.093349,0.006589,-0.055932,-0.000734,-0.118557,0.064746,-0.064746,-0.288585,1.000000,-0.565210
Pclass_3,-0.322308,-0.281004,0.092548,0.015790,-0.413333,-0.137143,0.137143,-0.626738,-0.565210,1.000000


D'après la matrice de corrélation des covariables, on observe que la survie est positivement corrélée au sexe féminin (et négativement au masculin), au nombre de parents/enfants à bord, et à la tarif du ticket. 
On observe aussi que la survie est positivement corrélée à tous les classes sauf la $3$ème, i.e, la classe économique. 

### On procède maintenant à la régression logistique
### 5. On inclut d'abord tous les paramètres dans la régression. Calculer les intervalles de confiance à $95\%$ pour l'ensemble des paramètres. Qu'observe-t-on ?

In [6]:
df = données

res = smf.logit(formula="Survived ~ Age + SibSp + Parch + Fare + C(Sex) + C(Pclass)", data=df).fit()
t = res.summary()
t
#αΣβ

Optimization terminated successfully.
         Current function value: 0.442576
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      883
Method:                           MLE   Df Model:                            7
Date:                Fri, 22 Oct 2021   Pseudo R-squ.:                  0.3354
Time:                        18:57:07   Log-Likelihood:                -394.34
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 6.452e-82
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          3.8409      0.447      8.602      0.000       2.966       4.716
C(Sex)[T.male]    -2.7609      0.199    -13.856      0.000      -3.151      -2.370
C(Pclass)[T.2]    -1.0231      0.294     -3.481      0.000      -1.599      -0.447
C(Pclass)[T.3]    -2.1499      0.290     -7.423      0.000      -2.718      -1.582
Age               -0.0395      0.008     -5.035      0.000      -0.055      -0.024
SibSp             -0.3501      0.110     -3.194      0.001      -0.565      -0.135
Parch             -0.1133      0.118     -0.964      0.335      -0.344       0.117
Fare               0.0030      0.002      1.223      0.221      -0.002       0.008
==================================================================================
"""

Remarquez que les intervalles de confiance à $95 \%$ sont bien affichés dans les deux dernières colonnes du tableau. 

On observe d'abord que le coefficient associée à la variable *dummy* du sexe masculin est négative, et le même vaut pour les *dummy* répresentant les classes $2$ et $3$. 
De plus, on voit que la plupart des coefficients sont négatifs, i.e., leurs intervalles de confiance à $95 \%$ sont contenus dans $\mathbb{R}_{-}$, sauf par $\beta_{\texttt{Parch}}$ et $\beta_{\texttt{Fare}}$. 

### 6. Déterminer la $p$-valeur du test
$$H_0 : \beta_{\texttt{Parch}} = 0 \text{ , contre } H_1 : \beta_{\texttt{Parch}} \neq 0$$
### Que peut-on conclure ?

In [7]:
pvaleur = float(t.tables[1].data[7][4])
print(f"p = {pvaleur:.2f} %")

p = 0.34 %


Puisque la $p$-valeur obtenue est supérieur au niveau $\alpha = 5 \%$ du test, la conclusion est qu'on **n'a pas d'évidences suffisantes pour rejetter l'hypothèse nulle**. Autrement dit, c'est raisonnable de considérer que $\beta_{\texttt{Parch}} = 0$. 

### 7. On reprend l'analyse en éliminant la variable ```Parch```. Calculer les intervalles de confiance à $95\%$ pour l'ensemble des paramètres. Qu'observe-t-on ?

In [8]:
df = données

res = smf.logit(formula="Survived ~ Age + SibSp + Fare + C(Sex) + C(Pclass)", data=df).fit()
t = res.summary()
t
#αΣβ

Optimization terminated successfully.
         Current function value: 0.443106
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      884
Method:                           MLE   Df Model:                            6
Date:                Fri, 22 Oct 2021   Pseudo R-squ.:                  0.3346
Time:                        18:57:07   Log-Likelihood:                -394.81
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 1.210e-82
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          3.8158      0.444      8.595      0.000       2.946       4.686
C(Sex)[T.male]    -2.7245      0.195    -13.981      0.000      -3.106      -2.343
C(Pclass)[T.2]    -1.0494      0.292     -3.596      0.000      -1.621      -0.477
C(Pclass)[T.3]    -2.1818      0.287     -7.614      0.000      -2.743      -1.620
Age               -0.0392      0.008     -5.013      0.000      -0.055      -0.024
SibSp             -0.3783      0.106     -3.561      0.000      -0.587      -0.170
Fare               0.0025      0.002      1.066      0.286      -0.002       0.007
==================================================================================
"""

Remarquez que les intervalles de confiance à $95 \%$ sont bien affichés dans les deux dernières colonnes du tableau. 

On observe cette fois que la variable ```Fare``` est la seule dont le coefficient présente un intervalle de confiance autour de $0$. Cela nous motive à nous poser la question si $\beta_{\texttt{Fare}}$ n'est bien égal à $0$. 

### 8. Déterminer la $p$-valeur du test

$$H_0 : \beta_{\texttt{Fare}} = 0 \text{, contre } H_1 : \beta_{\texttt{Fare}} \neq 0$$

In [9]:
pvaleur = float(t.tables[1].data[7][4])
print(f"p = {pvaleur:.2f} %")

p = 0.29 %


Encore une fois, on trouve une $p$-valeur supérieure au niveau du test, donc on **ne peut pas rejetter l'hypothèse nulle**, c'est-à-dire, il est raisonnable de considérer $\beta_{\texttt{Fare}} = 0$ et éliminer la variable ```Fare``` du modèle. 

### 9. On reprend l'analyse en éliminant la variable ```Parch``` et ```Fare```. Calculer les intervalles de confiance à $95\%$. Que peut-on conclure ?

In [10]:
df = données

res = smf.logit(formula="Survived ~ Age + SibSp + C(Sex) + C(Pclass)", data=df).fit()
t = res.summary()
t
#αΣβ

Optimization terminated successfully.
         Current function value: 0.443793
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      885
Method:                           MLE   Df Model:                            5
Date:                Fri, 22 Oct 2021   Pseudo R-squ.:                  0.3336
Time:                        18:57:08   Log-Likelihood:                -395.42
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 2.366e-83
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          4.0274      0.400     10.072      0.000       3.244       4.811
C(Sex)[T.male]    -2.7402      0.194    -14.110      0.000      -3.121      -2.360
C(Pclass)[T.2]    -1.1898      0.262     -4.544      0.000      -1.703      -0.677
C(Pclass)[T.3]    -2.3478      0.243     -9.668      0.000      -2.824      -1.872
Age               -0.0399      0.008     -5.111      0.000      -0.055      -0.025
SibSp             -0.3583      0.104     -3.437      0.001      -0.563      -0.154
==================================================================================
"""

Cette fois-ci on remarque que pour chacun des paramètres, les tests de nullité (comme les tests qu'on a méné pour ```Parch``` et ```Fare```) donnent tous une $p$-valeur plus petite que le niveau des tests. Ainsi, on ne peut supprimer aucune de ces variables du modèle. 

### 10. Dans ce modèle, quelles sont les probabilités de survie d'un homme dans un cas, et d'une femme dans un autre cas, chacun âgé de $22$ ans sans famille et voyageant en $1$ère classe ?

In [11]:
φ = lambda t: 1.0/(1.0 + np.exp(-t))

homme = np.array([1, 1, 0, 0, 22, 0])
femme = np.array([1, 0, 0, 0, 22, 0])

ph, pf = φ(np.dot(res.params, homme)), φ(np.dot(res.params, femme))

display(Latex(f"Probabilités de Survie de l'homme: ${100*ph:.2f} \%$"))
display(Latex(f"Probabilités de Survie de la femme: ${100*pf:.2f} \%$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Étonnamment, il semble que les chances de survie pour Jack n'étaient pas si mauvaises, lorsque Rose, par contre, était presque sûre de survivre. 